In [1]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
drone = Tello()

In [ ]:
objective = "Find the person who is wearing a blue cap"

In [ ]:
drone.streamon()
drone.takeoff()

In [ ]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [ ]:
def explore(drone):
    pass